# Importa los módulos necesarios y establece configuraciones

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import funs
import seaborn as sns
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.preprocessing import StandardScaler
init_notebook_mode(connected=True)
pd.options.display.max_rows = 10

## Importa el .csv que contiene las mediciones de todos los nodos

In [ ]:
df = pd.read_csv('all_nodos.csv')
df

## Suprime las columnas que no se necesitan

In [ ]:
try:
    df.drop(columns=['T_NODO', 'TRY', 'FECHA', 'Carga', 'RSSI', 'T_SERVIDOR', 'TIPO_Nodo'], inplace = True)
except KeyError:
    pass
df

## Atrasa la columna HORA del mes mayo en tres unidades, la hora estaba adelantada para esos registros

In [ ]:
ind_rem = [[x,int(df.HORA[x])-3] for x in range(len(df)) if df.MES[x] < 6]
df.loc[pd.DataFrame(ind_rem).iloc[:,0],'HORA'] = pd.DataFrame(ind_rem).iloc[:,1].values
ind_rem = [[x, df.HORA[x]+24,df.DIA[x]-1] for x in range(len(df)) if df.HORA[x]<0]
df.loc[pd.DataFrame(ind_rem).iloc[:,0],'HORA'] = pd.DataFrame(ind_rem).iloc[:,1].values
df.loc[pd.DataFrame(ind_rem).iloc[:,0],'DIA'] = pd.DataFrame(ind_rem).iloc[:,2].values

## Cambia los nombres de las columnas

In [ ]:
old_names = ['id_Variable','id_nodo','Temperatura','Humedad','Hum. Tierra','Nivel_UV','INT. LUMÍN'] 
new_names = ['id_Variable','NODO','TEMPERATURA','HUMEDAD RELATIVA','HUMEDAD DE LA TIERRA','NIVEL UV','INTENSIDAD LUMÍNICA']
df.rename(columns=dict(zip(old_names, new_names)), inplace=True)
df

## Se introduce un cero antes de las unidades de las columnas MES, DIA, HORA, MINUTO

In [ ]:
ind_rem = {}
for i in ['MES','DIA','HORA','MINUTO']:
    ind_rem[i] = [[x,'0'+str(df[i][x])] for x in range(len(df)) if df[i][x] < 10]
    df.loc[pd.DataFrame(ind_rem[i]).iloc[:,0],i] = pd.DataFrame(ind_rem[i]).iloc[:,1].values
df

## Se crea una columna con la estrucutura AÑO+MES+DIA+HORA+MINUTO

In [ ]:
df['TIME'] = [str(df.AÑO[i])+str(df.MES[i])+str(df.DIA[i])+str(df.HORA[i])+str(df.MINUTO[i]) for i in range(len(df))]
df.TIME

## Convierte la columna TIME al fomato Timestamp

In [ ]:
df['TIME'] = [pd.Timestamp(x) for x in df['TIME']]
df

## Suprime las columnas que no se van a utilizar

In [ ]:
try:
    df.drop(columns = ['id_Variable','AÑO','MES','DIA','MINUTO'], inplace = True)
except KeyError:
    pass

## Elimina los elementos de df anteriores al registro 2882, correspondientes a pruebas

In [ ]:
if len(df) == 50501:
    df = df.iloc[2822::,:].reset_index(drop = True)

In [ ]:
df.index = df.TIME
df.drop(columns = 'TIME', inplace = True)
df

## Mormaliza las variables

In [ ]:
df2 = StandardScaler().fit_transform(df.iloc[:,1:6])
df2 = pd.DataFrame(df2)
df2.columns = df.iloc[:,1:6].columns
df2.index = df.index
df.iloc[:,1:6] = df2
df

In [ ]:
# convierte a entero los registros de la columna HORA
df.HORA = [int(df.HORA[x]) for x in range(len(df))]

In [ ]:
n = df.NODO.value_counts()
# imprime los nodos contenidos en df y la cantidad de registros que posee
print(n)
nodos  = n.index.tolist()

## Separa los nodos

In [ ]:
tablas = {}
for i in nodos:
    tablas[i] = (df.query('NODO == %s' %i)).copy()#.reset_index(drop=False))
    
tabla = []
for i in nodos:
    tabla_nodo = df.query('NODO == %s' %i).copy()#.reset_index(drop=False)
    tabla_nodo.columns = [j+str(tabla_nodo.NODO[0]) for j in tabla_nodo.columns]
    tabla_nodo.drop(columns = ['NODO%s'%i],inplace=True)
    tabla.append(tabla_nodo)

## Realiza una gráfica interactiva de valores medidos por un nodo

### "tablas" es un diccionario de DFs. Para acceder al DF correspondiente a cada nodo se hace tabla[nodo], tabla[2] (nodo 2).
#### Cada DF tiene 7 columnas:
0 - TEMPERATURA  
1 - HUMEDAD RELATIVA  
2 - HUMEDAD DE LA TIERRA  
3 - HUMEDAD DE LA TIERRA  
4 - NIVEL UV  
5 - INTENSIDAD LUMÍNICA  
6 - HORA  
7 - TIME  

In [ ]:
for nodo in nodos:
    data = ([{'x': tablas[nodo].index,'y': tablas[nodo][col],'name': col}
             for col in tablas[nodo].columns[1:6]])
    layout = dict(title = 'NODO %s' %nodo,
                  yaxis = dict(title = 'MAGNITUD'),
                  xaxis = dict(title = 'FECHA')
                 )
    fig = dict(data=data, layout=layout)
    iplot(fig,layout)

## Se presentan los periodos de medición observados para cada nodo, calculados para separaciones de más de 60 minutos entre los registros


In [ ]:
# Aplica la función separa_nodos, que calcula el tamaño de los dataframes entre periodos de medición con separación #edge"
for i in nodos:
    print('NODO %s' %i)
    print(funs.separa_datos(tablas[i],60))
    
                    

## Apartir de la información anterior se tiene que:  
- ### los nodos 2, 3, 5 estuvieron funcionando en tres periodos de tiempo consecutivos.
### Dicha simultaneidad se presenta mayormente entre las fechas:  
- 2018-05-09 10:26:00 2018-05-10 14:37:00  
- 2018-05-23 10:09:00 2018-05-26 07:01:00  
- 2018-06-05 10:22:00 2018-06-08 18:40:00
- ### El nodo 8 comenzó a funcionar desde el 24 de mayo y apartir de ese peridodo sus mediciones concuerdan con algunos periodos de medición de los nodos 2, 3 y 5.  
  
### la simultaneidad  de mayor rango se presenta entre las fechas:
- 2018-05-24 09:42:00 2018-05-26 07:01:00  
- ### Los nodos 6 y 8 presentan la menor cantidad de registros y se encuentran en fechas dispares a las de los demás nodos.

## Concatena los DFs contenidos en tabla

In [ ]:
tabla = pd.concat(tabla).sort_index()
tabla

## Ordena las columnas por orden alfabético

In [ ]:
tabla = tabla.reindex(columns = sorted(tabla.columns))

## Matrices de correlación entre variables de diferentes nodos

In [ ]:
tcorr = tabla.corr()
for i in range(0,len(tcorr),6):
    for j in range(0,len(tcorr),6):
        sns.heatmap(tcorr[i:i+6].iloc[:,j:j+6], annot=True, cmap='RdYlGn',fmt=".2f")
        plt.show()

In [ ]:
# presentación gráfica de las variables entre los nodos, pero con matplotlib :v
# nodos = 2;3;4;5;6;8
for nodo in nodos:
    ejey = tablas[nodo].columns[1:6]
    tablas[nodo].plot(y = ejey, figsize=[20,5],title='Nodo %d'%nodo)
    plt.tight_layout()

In [ ]:
# grafica una variable para dos nodos y calcula la correlación de los dos nodos respecto a esa variable.
variable = tablas[2].columns[1]
plt.figure(figsize = [20,5])
nodo1 = 6
nodo2 = 8
for nodo in [nodo1, nodo2]:
    plt.plot(tablas[nodo].TIME,tablas[nodo][variable])
ncorr = pd.concat([tablas[nodo1].TEMPERATURA,tablas[nodo2].TEMPERATURA],axis=1).corr()
corr = ncorr.values[1][0]
plt.title('%s DE LOS NODOS %d Y %d'%(variable, nodo1, nodo2))
print('CORRELACIÓN DE %s EN LOS NODOS %d Y %d: %s'%(variable, nodo1,nodo2,corr))